# Dependencies

In [1]:
# --- Dependencies ---
import pickle
import numpy as np

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
with open('train.pkl', 'rb') as f:
  train = pickle.load(f)
with open('validation.pkl', 'rb') as f:
  validation = pickle.load(f)
with open('test.pkl', 'rb') as f:
  test = pickle.load(f)

In [3]:
X_train = [sequence for sequence, target in train]
X_val = [sequence for sequence, target in validation]
X_test = [sequence for sequence, target in test]

Y_train = [target for sequence, target in train]
Y_val = [target for sequence, target in validation]
Y_test = [target for sequence, target in test]

In [4]:
INPUT_SIZE = len(X_train[0][0])

# Dataset / DataLoader

In [5]:
class ReviewDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)
        
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]).float(), torch.tensor(self.y[idx]).float()

train_dataset = ReviewDataset(X_train, Y_train)
val_dataset = ReviewDataset(X_val, Y_val)
test_dataset = ReviewDataset(X_test, Y_test)

In [6]:
# Pad the sequences
def collate_fn(batch):
    inputs = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    
    # Pad sequences so they all have the same length within one batch
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)

    return inputs_padded, torch.stack(targets)

In [7]:
batch_size = 64
train_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1)
test_loader = DataLoader(test_dataset, batch_size=1)

# Model

In [8]:
class GRUModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, dropout):
    super(GRUModel, self).__init__()

    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.dropout = dropout

    # Define GRU layer
    self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    # Define output layer
    self.out = nn.Linear(hidden_size, 2)  # Two output nodes for λ and k
    #self.relu = nn.ReLU()

  def forward(self, x):
    # Forward propagate GRU
    y, hidden = self.gru(x)  # shape = (batch_size, seq_length, hidden_size)

    # Decode the hidden state of the last time step
    out = self.out(y[:, -1, :])
    #out = self.relu(out)

    # Get parameters of the Weibull distribution
    #λ = torch.exp(out[:, 0])
    λ, k = torch.exp(out[:, 0]), torch.exp(out[:, 1])  # exp to ensure positive values
    #λ, k, z1 = torch.exp(out[:, 0]), torch.exp(out[:, 1]), torch.exp(out[:, 2])
    return λ, k

In [9]:
def weibull_survival_function(λ, k, t):
  # small constant to prevent division by zero
  epsilon = 1e-7
  λ = torch.clamp(λ, min=epsilon)
  return torch.exp(-(t / λ) ** k)

# Train

In [10]:
device = torch.device('cuda')

In [11]:
from sklearn.metrics import roc_auc_score, fbeta_score, accuracy_score, precision_score, recall_score

def calculate_metrics(model, loader, device, threshold, metrics, print_results=False):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            λ, k = model(inputs)
            review_time = targets[:, 0]
            remembered = targets[:, 1]
            predicted_survival = weibull_survival_function(λ, k, review_time)

            preds = predicted_survival
            all_preds.append(preds.cpu().numpy())
            all_targets.append(remembered.cpu().numpy())

    # Flatten the lists of arrays for computing metrics
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    
    threshold_preds = (all_preds > threshold).astype(int)

    results = {}
    for metric in metrics:
        if metric == "roc_auc":
            result = roc_auc_score(all_targets, all_preds)
        elif metric == "f2":
            result = fbeta_score(all_targets, threshold_preds, beta=2)
        elif metric == "accuracy":
            result = accuracy_score(all_targets, threshold_preds)
        elif metric == "precision":
            result = precision_score(all_targets, threshold_preds)
        elif metric == "recall":
            result = recall_score(all_targets, threshold_preds)
        else:
            raise ValueError(f"Unsupported metric: {metric}")
        
        results[metric] = result
    if print_results:
        print(results)
    return results


In [12]:
def log_scores(epoch_scores, score_dict=None):
    """
    This function takes the epoch scores and a dictionary of previous scores (or None),
    and returns an updated dictionary of scores.
    """

    if score_dict is None:
        # Initialize score_dict with empty lists if it's None
        score_dict = {key: [] for key in epoch_scores.keys()}

    for key in epoch_scores.keys():
        if key in score_dict:
            score_dict[key].append(epoch_scores[key])
        else:
            score_dict[key] = [epoch_scores[key]]

    return score_dict

In [13]:
def train(*, epochs = 5, hidden_size = 128, num_layers = 2, lr = 0.001, dropout = 0.2, weight_decay = 0):
  model = GRUModel(INPUT_SIZE, hidden_size, num_layers, dropout)  # Adjust the parameters as needed
  model = model.to(device)
  
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)  # Adjust the learning rate as needed
  criterion = nn.BCELoss()

  scores = {}
  for epoch in range(epochs):
      model.train()

      train_loss = 0.0

      progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
      for inputs, targets in progress:
          inputs, targets = inputs.to(device), targets.to(device)
          
          λ, k = model(inputs)
          review_time = targets[:, 0]
          remembered = targets[:, 1]
          
          predicted_survival = weibull_survival_function(λ, k, review_time)

          loss = criterion(predicted_survival, remembered)

          optimizer.zero_grad()

          # clip grads
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

          loss.backward() 
          optimizer.step()

          train_loss += loss.item()
          progress.set_postfix({'loss': train_loss / (progress.n + 1)})

      epoch_loss = train_loss / len(train_loader)
      print(epoch_loss)

      if epoch % 2 == 0:
        #epoch_scores = calculate_metrics(model, train_loader, device, metrics=["roc_auc", "recall", "precision"], print_results=True)
        epoch_scores = calculate_metrics(model, val_loader, device, threshold=0.6, metrics=["roc_auc"], print_results=True)
        #epoch_scores = calculate_metrics(model, val_loader, device, threshold=0.7, metrics=["roc_auc", "recall", "precision"], print_results=True)
        # epoch_scores = calculate_metrics(model, val_loader, device, threshold=0.8, metrics=["roc_auc", "recall", "precision"], print_results=True)
        # epoch_scores = calculate_metrics(model, test_loader, device, metrics=["roc_auc", "recall", "precision"], print_results=True)
        scores = log_scores(epoch_scores, scores)

  return scores

In [14]:
def print_max_values(input_dict):
    for outer_key, inner_dict in input_dict.items():
        print(outer_key)
        for inner_key, values_list in inner_dict.items():
            max_value_index = np.argmax(values_list)
            max_value = values_list[max_value_index]
            print(f'{inner_key}: {round(max_value, 5)} (at {max_value_index})')

In [16]:
  # Set up logging
from itertools import product

# Define the grid of hyperparameters
grid = {
    'lr': [0.001],
    'hidden_size': [128],
    'num_layers': [1, 2, 3],
    'dropout': [0.5],
    'weight_decay': [0.1, 0.01, 0.001],
}

all_results = {}

for hyperparams in product(*grid.values()):
    lr, hidden_size, num_layers, dropout, weight_decay = hyperparams
    if num_layers == 1 and dropout:
      continue
    desc = f"lr={lr}, hidden={hidden_size}, nl={num_layers}, dropout={dropout}, decay={weight_decay}"

    results = train(epochs=40, hidden_size=hidden_size, num_layers=num_layers, lr=lr, dropout=dropout)
    all_results[desc] = results

    print_max_values(all_results)

    # Update the best hyperparameters and the best score if the current score is higher
    #print(score)
    #if score > best_score:
     #   best_hyperparams = hyperparams
       # best_score = score

# Print the best hyperparameters and the best score
#print(f"Best Hyperparameters: {best_hyperparams}")
#print(f"Best ROC AUC Score: {best_score}")

Epoch 1/40: 100%|██████████| 416/416 [00:03<00:00, 107.77it/s, loss=0.401]


0.3908064153690178
{'roc_auc': 0.5886040648046369}


Epoch 2/40: 100%|██████████| 416/416 [00:03<00:00, 124.99it/s, loss=0.371]


0.3682910423869124


Epoch 3/40: 100%|██████████| 416/416 [00:03<00:00, 125.15it/s, loss=0.374]


...

0.35038532119674176


Epoch 39/40: 100%|██████████| 416/416 [00:04<00:00, 101.54it/s, loss=0.354]


0.3494231758209375
{'roc_auc': 0.6744446039082402}


Epoch 40/40: 100%|██████████| 416/416 [00:03<00:00, 105.94it/s, loss=0.348]

0.3474076883509182
lr=0.001, hidden=128, nl=2, dropout=0.5, decay=0.1
roc_auc: 0.69864 (at 9)
lr=0.001, hidden=128, nl=2, dropout=0.5, decay=0.01
roc_auc: 0.69062 (at 5)
lr=0.001, hidden=128, nl=2, dropout=0.5, decay=0.001
roc_auc: 0.69032 (at 12)
lr=0.001, hidden=128, nl=3, dropout=0.5, decay=0.1
roc_auc: 0.68908 (at 7)
lr=0.001, hidden=128, nl=3, dropout=0.5, decay=0.01
roc_auc: 0.6819 (at 3)
lr=0.001, hidden=128, nl=3, dropout=0.5, decay=0.001
roc_auc: 0.69462 (at 9)
